In [1]:
import random
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import math
import seaborn as sns
from sklearn.model_selection import GridSearchCV,ParameterGrid,cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import train_test_split,StratifiedKFold,learning_curve,KFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.pipeline import Pipeline

# 导入算法库
from sklearn.linear_model import LinearRegression,Ridge,LassoCV,BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBClassifier,XGBRegressor
# import xgboost as xgb
# from xgboost import plot_importance
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from random import shuffle
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import svm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
from matplotlib.font_manager import * 
plt.style.use('ggplot')          # 配色
matplotlib.rcParams['figure.figsize'] = [20, 6] # for square canvas
# sns.set(font='SimHei')  # 解决Seaborn中文显示问题
pd.set_option('display.max_colwidth', 500)       # 设置字符串显示长度
from IPython.core.interactiveshell import InteractiveShell
from sklearn.impute import SimpleImputer
InteractiveShell.ast_node_interactivity = 'all'
#指定默认字体
myfont = FontProperties(fname='./fonts/simhei.ttf') 
matplotlib.rcParams['font.sans-serif'] = ['SimHei'] 
matplotlib.rcParams['font.family']='sans-serif'
%config IPCompleter.greedy=True

In [2]:
data=pd.read_pickle('./data/diag_all.pkl')

In [3]:
data.诊断类型.value_counts()
sum(data.病人ID.duplicated())
data.shape

阳性体征    174570
疾病诊断     27633
Name: 诊断类型, dtype: int64

157824

(202203, 19)

In [ ]:
data_ino=pd.read_csv('./data/Medical_INO_details_Information.csv',encoding='gbk')

In [ ]:
data.shape
data_ino.info()

In [ ]:
# data.head()

### 在Excel（后面发现Excel未完全显示）中观察体检数据表INO，总结以下：
- 1. 诊断标志9 表示**无异常**，只有一个编号759230的体检记录中 一个男的异常提示被记录为h（其他为空白）， 但是阳性为0， 考虑为录入失误, 将'诊断标志'修正为4
  2. 诊断标志4 表示**异常**(h/阳性),除了三个：其中一个编号820410的体检记录， 一个男的异常记录为z，阳性为1， 考虑录入失误， 将'异常提示'修正为h 
  3. 诊断标志3 表示**无异常**(z/心率)   考虑转化为0
  4. 诊断标志2 表示**异常**(h)
  5. 诊断标志1 表示**异常**(l)/**心率**（无具体数值，并且阳性特征全部为0）
  6. 诊断标志0 表示**无异常**(空白)，除了一个编号759224的体检记录,异常记录为'h',阳性为0， 录入失误， 将诊断标志修正为**2**

In [ ]:
data_ino.groupby('诊断标志')['异常提示'].value_counts().unstack()

In [ ]:
data_ino.loc[759224, '诊断标志'] = 2                 # 修正 #6

data_ino.at[(data_ino.异常提示 == 'z')&(data_ino.诊断标志 == 4), '异常提示'] = 'h' # 修正 #2

data_ino.loc[759230, '诊断标志'] = 4              # 修正 #1

data_ino['诊断标志'] = np.floor(data_ino['诊断标志'])

data_ino['诊断标志'] = data_ino.apply(lambda x: 0 if x['诊断标志'] == 3 or x['诊断标志'] > 4 else x['诊断标志'], axis=1)

In [ ]:
data_ino['诊断标志'].value_counts()

In [ ]:
data_ino.groupby('异常标志')['异常提示'].value_counts().unstack()            ## 异常标志这个 0 值有点问题， 不可信
# data_ino[(data_ino.异常提示 == 'z')&(data_ino.异常标志 == 1)]

In [ ]:
temp_ino = data_ino
# 函数 填充'诊断标志'为NaN的情况 为 透视表 做准备
def fill_labels(df):
    if pd.isnull(df['诊断标志']):
        if pd.isnull(df['正常结果']):
            if df['异常提示'] == 'z':
                return 0
            elif df['异常提示'] == '+' or df['异常提示'] == 'h':
                return 2
            elif df['异常提示'] == '-' or df['异常提示'] == 'l':
                return 1
        else:
            return 0        # 正常全为0
    else:
        return df['诊断标志']
# temp_ino.at[temp_ino['诊断标志'].isnull() & -(temp_ino['正常结果'].isnull()) & (temp_ino['', '诊断标志'] = 0
temp_ino.groupby('诊断标志')['异常提示'].value_counts().unstack()                     # 处理前
temp_ino['诊断标志'] = temp_ino.apply(fill_labels, axis=1)
temp_ino.groupby('诊断标志')['异常提示'].value_counts().unstack()                      # 处理后

但是 比如9.0 或者其他值，并且'异常提示'为NaN的 没有上榜。。。 比如'鼻','心率'

In [ ]:
# data_ino.groupby(['病人ID'])
combined_ino = temp_ino.pivot_table(values='诊断标志', index='病人ID', columns='项目名称') #, aggfunc=lambda x : (label_Dict[x]))

In [ ]:
for col in combined_ino.columns:
    combined_ino[col].value_counts()
# combined_ino['血压'].value_counts()

完善血压 心率 年龄 性别

血压需要处理数值
- 高血压： 18岁成人，收缩压 >= 140 **或** 舒张压 >= 90 （单位mmHg
- 低血压： 血压低于**90/60**mmHG

In [ ]:
data_press = data_ino.loc[data_ino['项目名称'] == '血压', ['病人ID', '检查结果']]
# pattern_pa
def fill_press(df):
    pattern_press = re.compile(r"(?P<收缩压>\d+)/(?P<舒张压>\d+)")
#     res = 
# data_press.apply()
pa = r"(\d+)/(\d+)"
pa_name = r"(?P<收缩压>\d+)/(?P<舒张压>\d+)"
# data_press[['收缩压', '舒张压']] = data_press.检查结果.str.extract(pa)
press_re = data_press.检查结果.str.extract(pa_name)                                   # extract() 与 extractall() 
press_res = pd.concat([data_press, press_re], axis=1).set_index('病人ID')                           # 前者找第一个匹配 后者找全部匹配（所以有match）
press_res.drop(['检查结果'], axis=1, inplace=True)
# data_press

心率的 诊断标志有问题。。。

In [ ]:
# sum(res.isnull().sum())
# res == combined_ino_temp
# res.isnull()
# res.loc[res.index.isin(['3000045556']), 'ABO血型'] = 2
# res

# combined_ino_res['心率'].value_counts()#.at[:, '体检总结'] = data_report['体检总结']                #['肌酐(Cr)'] #体检总结
# sample = combined_ino_temp.sample(1) #.index.is_unique
# data_report = data_report.drop_duplicates()
# data_report.index.is_unique
# sample
def heart_beat_float(df):
#     pattern = re.compile(r"*?(\d+)*?")
#     df['检查结果'] = float(re.match(pattern, df['检查结果'].strip(' ')))
#     df['检查结果'].strip(' ')
    if type(df['检查结果']) != float:
        df['检查结果'].strip(' ')
    return df
        
# beat_pattern = re.compile(r"()")
heart_beat = data_ino.loc[data_ino['项目名称'] == '心率', ['病人ID', "检查结果"]].set_index('病人ID')
# heart_beat = heart_beat.apply(heart_beat_float, axis=0)
# heart_beat.drop(type(heart_beat['检查结果'])!=float)
# type(heart_beat['检查结果'])!=float
# heart_beat.set_index('病人ID')
#  利用掩码查看异常值 发现有奇怪文本“拒绝检查” 和 6 "61 61" "正常" 这样的异常数字
mask = heart_beat.loc[:, ['检查结果']].applymap(lambda x: isinstance(x, str) and (len(x) == 2 or len(x) == 3) and x.isdigit()).values
# mask = heart_beat.loc[:, ['检查结果']].applymap(lambda x: isinstance(x, int)).values
heart_beat.loc[:, ['检查结果']] = heart_beat.loc[:, ['检查结果']].applymap(lambda x: float(x) if isinstance(x, str) and (len(x) == 2 or len(x) == 3) and x.isdigit() else np.nan)

beat_press = pd.merge(heart_beat[mask], press_res, on='病人ID', how='left')
beat_press.rename(columns={'检查结果':"心率值"}, inplace=True)
# beat_press.drop_duplicates(inplace=True)
beat_press = beat_press[~beat_press.index.duplicated(keep='first')]            # 删去重复的病人ID（index)
sum(beat_press.index.duplicated())
beat_press.info()

In [ ]:
# df = pd.DataFrame([[1, None], [5, 4], [4, 4]], columns=['a', 'b'])
# def apply_test(input):
#     if input.a == 1:
#         input['b'] = 3
#     return input
# def test2(input):
#     if input.isnull().sum() > 0:
#         df.drop(input)
#     return input
# sum(df.isnull().sum())
# # test_res = df.apply(test2, result_type='reduce')
# # sum(test_res.isnull().sum())
# # sum(df.isnull().sum())
# df
# test_res

# df.info()

In [ ]:
data_report = data_ino[['病人ID', '年龄', '性别' ,'体检总结']].copy().set_index('病人ID')
# data_report = data_report.drop_duplicates()            ## 去重没有用？？？ 病人ID依然重复
data_report = data_report[~data_report.index.duplicated(keep='first')]

combined_ino_temp = combined_ino.copy()
combined_ino_temp = pd.merge(combined_ino_temp, data_report, on='病人ID', how='left', left_index=True, sort=False)

pattern_UP = re.compile(r' (?P<项目名称>[^↓\n]+?)：[^↓]+? ↑')     # 非贪婪匹配 ? ; 匹配1+次 + ; 匹配非符号[^]
pattern_DOWN = re.compile(r' (?P<项目名称>[^↑\n]+?)：[^↑]+? ↓')     # 非贪婪匹配 ? ; 匹配1+次 + ; 匹配非符号[^]

def fill_Combined_re(combined_ino, temp, pattern_UP=pattern_UP, pattern_DOWN=pattern_DOWN):
#     pattern_UP = r' (?P<项目名称>[^↓\n]+?)：[^↓]+? ↑'
#     item_UP = combined_ino.体检总结.str.extractall(pattern_UP)
#     list_UP = list(item_UP.项目名称.map(lambda x: x.strip()))
    combined_ino
    UP = pattern_UP.findall(str(combined_ino.体检总结))
    list_UP = list(map(lambda x: x.strip(), UP))
    for item in list_UP:
        if item in temp.columns:
#             if combined_ino[item] != 2:
#                 print('病人: ', item, " 上升值 :", combined_ino[item])
#                 print(item, ':', combined_ino.item)
#             print(item)
            combined_ino[item] = 2
#             temp.loc[temp.index.isin([combined_ino.index[0]]), item] = 2
#         else:
#             print(item)
    
#     pattern_DOWN = r' (?P<项目名称>[^↑\n]+?)：[^↑]+? ↓'
#     item_DOWN = combined_ino.体检总结.str.extractall(pattern_DOWN)
#     list_DOWN = list(item_DOWN.项目名称.map(lambda x: x.strip()))
    DOWN = pattern_DOWN.findall(str(combined_ino.体检总结))
    list_DOWN = list(map(lambda x: x.strip(), DOWN))
    for item in list_DOWN:
        if item in temp.columns:
            temp_value = combined_ino[item]
            combined_ino[item] = 1
#             if temp_value != 1:
#                 print('病人: ', item, " 下降值 :", combined_ino[item])
#             temp.loc[temp.index.isin([combined_ino.index[0]]), item] = 1
#         else:
#             print(item)
    return combined_ino

# combined_ino_temp = combined_ino_temp.sample(1000)
sum(combined_ino_temp.isnull().sum())
combined_ino_res = combined_ino_temp.copy()
res = combined_ino_temp.apply(fill_Combined_re, temp=combined_ino_res, axis=1)  #  apply 到每一行
sum(combined_ino_temp.isnull().sum())
sum(combined_ino_res.isnull().sum())
sum(res.isnull().sum())

测试正则表达式的正确性（似乎会漏掉NAN的数据，因为我感觉不止三个）

In [ ]:
# pattern_UP = r' (?P<项目名称>[^↓\n]+?)：[^↓]+? ↑'
# item_UP = test_src.str.extractall(pattern_UP)
# pattern_DOWN = r' (?P<项目名称>[^↑\n]+?)：[^↑]+? ↓'
# item_DOWN = test_src.str.extractall(pattern_DOWN)
# print(item_UP)
# print(item_DOWN)
# pattern = re.compile(r' [^↓\n]+?：[^↓]+? ↑')
for i in range(111110):
    test_src = res.sample(1)
    pattern = re.compile(r' (?P<项目名称>[^↓\n]+?)：[^↓]+? ↑')     # 非贪婪匹配 ? ; 匹配1+次 + ; 匹配非符号[^]
    pattern_res = pattern.findall(str(test_src.体检总结))
    # pattern_res = pattern.findall(test_src.values[0])
    for item in list(map(lambda x: x.strip(), pattern_res)):
        if item in test_src.columns:
            if test_src[item].values != 2:
                print(test_src[item])            
# type(res)

In [ ]:
train_df = pd.merge(res, beat_press, on='病人ID', how='left')
# train_df

combined_res = train_df.copy()
patient_num = combined_res.shape[0]       # 样本总数
def cut(df, cutoff):
    total_row = df.shape[0]
    for col in df.columns:
        cnt = df[col].count()
        if (cnt / total_row) < cutoff:
            df.drop(col, axis=1, inplace=True) 
#         elif(cnt / total_row)
# combined_res.apply(lambda x : x.isnull().sum()).plot()
# cut(combined_res, 0.5)                   # 缺失值2/3以上就删掉
combined_res.dropna(axis=1, thresh=patient_num/3, inplace=True)

In [ ]:
combined_res.shape

In [ ]:
# plt.plot([-1,2,-5,3]) 
# plt.title(u'Linux公社的网址是www.linuxidc.com',fontproperties=myfont) 
# plt.show()

In [ ]:
import missingno
missingno.matrix(combined_res)

In [ ]:
gender_dict = {'男':1, '女':0}
combined_res['性别'] = combined_res['性别'].map(gender_dict)

In [ ]:
combined_res.loc[:, ['心率值']].describe()              # 最小心率 最大心率惨不忍睹
combined_res.loc[:, ['心率值']].median()            # 最小心率 最大心率惨不忍睹

查看每一个项目的 值分布

In [ ]:
imp_median = SimpleImputer(strategy="median")                 # 使用中位数填充
combined_temp = combined_res.drop(columns=['体检总结'], axis=1)
combined_temp.loc[:, :] = imp_median.fit_transform(combined_temp)

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OneHotEncoder
# combined_temp

combined_temp.loc[:, '年龄'] = np.where(combined_temp.loc[:, '年龄']>100,  combined_temp.loc[:, '年龄'].median(), combined_temp.loc[:, '年龄'])
combined_temp.loc[:, '心率值'] = np.where(combined_temp.loc[:, '心率值']>150,  combined_temp.loc[:, '心率值'].median(), combined_temp.loc[:, '心率值'])

# 连续变量编码： 年龄：kmeans分箱， 心率： 等位分箱（数量相等）
width_est = KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='kmeans')          
count_est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')         
age_dummies = combined_temp.loc[:, '年龄'].values.reshape(-1, 1) 
combined_temp.loc[:, '年龄'] = width_est.fit_transform(age_dummies)

heart_dummies = combined_temp.loc[:, '心率值'].values.reshape(-1, 1)
combined_temp.loc[:, '心率值'] = count_est.fit_transform(heart_dummies)

# Onehot-encode 性别（无关离散变量）
# gender_enc = OneHotEncoder()
# gender_bin = combined_temp.loc[:, '性别'].values.reshape(-1, 1)
# gender_mat = gender_enc.fit_transform(gender_bin).astype(int)
# gender_mat.columns = ['男', '女']
# gender_mat
# combined_temp = pd.concat([combined_temp, gender_mat], axis=1)

# # dummies
# gender_bin = pd.get_dummies(combined_temp['性别'])
# gender_bin.rename(columns={0:'男', 1:'女'}, inplace=True)
# combined_temp = pd.concat([combined_temp, gender_bin], axis=1)

# combined_temp = combined_temp.drop(columns=['性别'])         

## 计算健康分数的数学模型
参考金融风控领域的信用分的模型$^{【1】}$，评分卡的分值刻度通过将**分值**表示为**比率对数**的线性表达式:

$$Score = A-B*log(odds )$$  其中，A为补偿，B为刻度，都为常数 

$$odds = \frac{p}{1-p}$$

$$p = w_1X_1 + w_2X_2 + w_3X_3 +...+w_nX_n +b= W^{T}X+b$$

其中$p$由Lasso线性回归模型（与岭回归类似）得到，范围[0, 1]，表示非健康的概率。引入违约翻倍系数PDO，即当 **$odds$**（非健康：健康比例)为两倍的时候，分数为$Score+PDO$：

$$Score +PDO = A-B*log(2*odds)$$
### 计算参数A，B
   解两个方程，得$$B=\frac{PDO}{ln(2)}$$, $$A = Score_0 + B*ln(odds_0)$$
    
   这里的 **$odds_0$** 为基准坏/好比例，这里取诊断结果表的阳性/阴性比例，得到$\frac{19832}{25543}=0.776416$, 
    
   $ln(odds_0) = -0.253066$ 设置**基准分$Score_0$为70，$PDO=20$**（这里可以自行调整，为了将大部分的分数限制在100分以内，我使用这个参数）代入得：
   
   $$B = 28.853900； A = 62.698058$$
    
引用【1】：[信用评分卡模型开发与应用](shichen.name/slide/20171115scorecard/#60)
    
    【2】：基于XGBOOST的用户信用评分建模_韩修龙

In [ ]:
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA          ## PCA 无监督降维得到新的正交特征矩阵，模型可解释性 差

y_pred = KMeans(n_clusters=5).fit_predict(combined_temp)

# cluster_dict = {}
# for index, v in enumerate(y_pred):
#     cluster_dict[combined_temp.columns[index]] = v

pca = PCA(n_components=0.99)             # 不直接指定降维的维度，而指定降维后的主成分方差和 比例
train_input = combined_temp.loc[:, combined_temp.columns]   # 取 输入
pca.fit(train_input)
print(pca.explained_variance_ratio_)  # 代表降维后的各主成分的方差值。方差值越大，则说明越是重要的主成分 
print(pca.explained_variance_)        # 代表降维后的各主成分的方差值占总方差值的比例，这个比例越大，则越是重要的主成分。
print(pca.n_components_)

pca_res = pca.transform(train_input)
# pca_res.columns

# sum(combined_ino_res.isnull().sum())
# # combined_ino_res == combined_ino_temp
# combined_ino_temp.isnull().sum()

# sample.get('总胆红素(TBIL)')                   # 验证OK
# sample.get('总胆红素(TBIL)')                   # 验证OK
# sample.columns              # 验证OK
# combined_ino_temp.index.is_unique            # 病人ID重复 == 

# data_ino.info()
# data_ino[(data_ino.项目代码 == 1004)]# & (data_ino.检查结果 == '阴性')]     # 同一个项目代码对应不同的 项目名称 所以 检查结果 和 异常提示 都不一样

#data_ino[data_ino['项目名称'] == '血压']

# test_src = res.sample(1)
# # test_src.values[0]
# test_src

# # list(item_DOWN.项目名称.map(lambda x: x.strip()))
# test_src.index
# item_DOWN.index[0]

发现同样的**项目名称**居然对应不一样的**项目代码**， 同样的**项目代码** 对应两个**项目名称** ==

In [ ]:
data_ino.pivot_table(index=['项目代码', '项目名称'], columns='异常提示', values='诊断标志', aggfunc=len)

In [ ]:
data_ino_age = data_ino[['年龄', '病人ID', '性别']].copy()
data_ino_age_temp = data_ino_age.rename(columns = {'年龄': 'age', '性别': 'gender'})
data_ino_age_temp.loc[data_ino_age_temp.gender == '男', ['gender']] = 'male'               # sns 不支持中文，除非加载字体
data_ino_age_temp.loc[data_ino_age_temp.gender == '女', ['gender']] = 'female'
data_ino_age_temp = data_ino_age_temp.drop_duplicates()
plt.figure(figsize=(20, 7))
sns.violinplot(x='age', data=data_ino_age_temp, orient='h', y='gender', split=True, palette='Set2', fontproperties=myfont)
plt.title('gender-age')
plt.show()

男性最大197岁应该是录入错误

In [ ]:
# age = data_ino[['病人ID', '年龄']].groupby(['病人ID'])
data_ino_female = data_ino[data_ino.性别.isin(["女", ])]
data_ino_male = data_ino[data_ino.性别.isin(["男", ])]
age_female = data_ino_female[['年龄', '病人ID']]
age_male = data_ino_male[['年龄', '病人ID']]
# age_mask = age.duplicated()
age_dup_female = age_female.drop_duplicates()
age_dup_male = age_male.drop_duplicates()
# age_dup = age_dup[['病人ID', '年龄']].value_counts()

# age_dup_female.年龄.value_counts().sort_index().plot.bar(figsize = (20, 5))
# age_dup_male_count = age_dup_male.年龄.value_counts().sort_index()
# fig, axis1 = plt.subplots(1,1,figsize=(20,5))
# age_dup_male_count['age'] = age_dup_male_count.index
# age_dup_male_count['count'] = age_dup_male_count[age_dup_male_count['age']]
# age_dup_male_count_df = age_dup_male_count.to_frame()
# age_dup_male_count_df[age] = age_dup_male_count_df.index
# sns.barplot(age_dup_male_count_df.index, age_dup_male_count_df.年龄, data = age_dup_male_count)
g = sns.countplot(x='年龄', data = age_dup_male, palette='rainbow')
# g.set_axis_labels("age", "count")
# plt.show()
g1 = sns.countplot(x='年龄', data = age_dup_female, palette='ocean_r')
# g1.set_axis_labels('age', 'count')
# age_dup.年龄.value_counts()
# age_dup_male

### 引入诊断结果，转化为有监督回归问题

In [ ]:
disease_rate = data.groupby('诊断名称')['诊断类型'].value_counts().unstack().fillna(0)
# 发病人数：kmeans分箱
kmeans_bins = KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='kmeans')    
disease_col = disease_rate.loc[:, '疾病诊断'].values.reshape(-1, 1)

disease_rate.loc[:, 'label'] = kmeans_bins.fit_transform(disease_col)
disease_rate

### 在Excel（后面发现Excel未完全显示）中观察体检数据表INO，总结以下：
- 1. 诊断标志9 表示**无异常**，只有一个编号759230的体检记录中 一个男的异常提示被记录为h（其他为空白）， 但是阳性为0， 考虑为录入失误, 将'诊断标志'修正为4
  2. 诊断标志4 表示**异常**(h/阳性),除了三个：其中一个编号820410的体检记录， 一个男的异常记录为z，阳性为1， 考虑录入失误， 将'异常提示'修正为h 
  3. 诊断标志3 表示**无异常**(z/心率)   考虑转化为0
  4. 诊断标志2 表示**异常**(h)
  5. 诊断标志1 表示**异常**(l)/**心率**（无具体数值，并且阳性特征全部为0）
  6. 诊断标志0 表示**无异常**(空白)，除了一个编号759224的体检记录,异常记录为'h',阳性为0， 录入失误， 将诊断标志修正为**2**

标签：将诊断类型数为0的患者判断为健康，大于0的为非健康, 收缩压和舒展压的数值缺失较多

In [ ]:
check_temp = data.groupby('病人ID')['诊断类型'].value_counts().unstack()
check_temp.fillna(0, inplace=True)
check_temp['label'] = np.where(check_temp['疾病诊断'] == 0, 0, 1)
check_temp.drop(columns=['疾病诊断', '阳性体征'], axis=1, inplace=True)
train_x = combined_temp.copy()
# 把输入和label 合起来
train_input = pd.merge(train_x, check_temp, on='病人ID', how='left')
train_input.fillna(0, inplace=True)
train_input['label'].value_counts()
train_input

年龄之前的输入数据全部dummies化

In [ ]:
items = train_input.columns[:-6]
#  全部取 整数部分
floor_train = np.floor(train_input.loc[:, train_input.columns[:-6]])
# floor_train = train_x.copy()
# 对数据清洗 ，针对不同的情况 ： 正常、低、高进行dummies处理
for item in items:
#     floor_train.loc[:, item].value_counts()
#  Series.index .values
#     floor_train.loc[:, item].value_counts()
    # 表示在各个检查项目出现的值， 并且根据情况进行 dummies化
    index = floor_train.loc[:, item].value_counts().index
    index = set(index)
#     print(item, index)
    
    # 如果只有一个值 删除
    if len(index) == 1 or len(index) > 4:
        floor_train = floor_train.drop(columns = [item])  
#     elif len(index) 
    elif index == {0, 4}:
        floor_train[item] = np.where(floor_train[item] == 4, 1, 0)
    # 4 2 都表示阳性
    elif index == {0, 2, 4}:
        floor_train[item] = np.where(floor_train[item] == 4, 2, floor_train[item])
        floor_train[item] = np.where(floor_train[item] == 2, 1, 0)
    #  4 这里重新赋值为2 其实也有阳性的意思 只能取片面意思
    elif index == {0, 1, 2, 4}:
        floor_train[item] = np.where(floor_train[item] == 4, 2, floor_train[item])
        bins = pd.get_dummies(floor_train[item])
        bins.rename(columns={0: item+'_正常', 1: item+'_低', 2: item+'_高'}, inplace=True)   
        floor_train = pd.concat([floor_train, bins], axis=1)

        floor_train = floor_train.drop(columns = [item])  
    # 0 正常 1 低 2 高
    elif index == {0, 1, 2}:
        bins = pd.get_dummies(floor_train[item])
        bins.rename(columns={0: item+'_正常', 1: item+'_低', 2: item+'_高'}, inplace=True)   
        floor_train = pd.concat([floor_train, bins], axis=1)

        floor_train = floor_train.drop(columns = [item])  

# gender_bin = pd.get_dummies(combined_temp['性别'])
# gender_bin.rename(columns={0:'男', 1:'女'}, inplace=True)   
# combined_temp = pd.concat([combined_temp, gender_bin], axis=1)

# combined_temp = combined_temp.drop(columns=['性别'])  

In [ ]:
floor_train

In [ ]:
# train_input['QRS'].value_counts()

上文中分箱的连续变量：年龄，心率值

In [ ]:
age_gender_HRate = train_input.loc[:, ['性别', '年龄',  '心率值'] ]
# ss = StandardScaler()     # 根据方差来缩放，标准化
mm = MinMaxScaler()         #根据极大极小值，归一化
# X_train.loc[: ] = ss.fit_transform(X_train)
age_gender_HRate.loc[: ] = mm.fit_transform(age_gender_HRate)
age_gender_HRate

很好奇最后的结果和男女性别的关系。

结果是，女性不健康（和之前的结论相反，之前认为样本男性年龄大于女性有关）

In [ ]:
X_train

In [ ]:
# X_train = train_input.loc[ :, train_input.columns[:-1] ]   # 取 输入
X_train = pd.concat([floor_train, age_gender_HRate], axis=1)
y = train_input.loc[:, 'label']

model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(X_train, y) # 此处 alpha 为通常值 #fit 把数据套进模型里跑
coef = pd.Series(model_lasso.coef_, index = X_train.columns)             # .coef_ 可以返回经过学习后的所有 feature 的参数。
# 如果权值为 0 就是被筛选掉的 feature
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
for c in coef.index:
    print(c + " coef :" + str(coef[c]))
imp_coef = pd.concat([coef.sort_values().head(12), 
                     coef.sort_values().tail(12)])                       #选头尾各10条，.sort_values() 可以将某一列的值进行排序。
# matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
plt.figure(figsize=(20,15))
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model")   

预测每一个样本不健康的概率，根据公式算出来最后的分数

In [ ]:
predict_res = model_lasso.predict(X_train)
def calculate_score(prob):
    A = 62.698058
    B = 28.853900
    if prob == 1:
        return 100
    return A -  B * math.log(prob / (1 - prob))
score_list = list(map(calculate_score, predict_res))

In [ ]:
plt.figure(figsize=(20, 10))    # 指定画布尺寸
axis = sns.distplot(a=score_list, color='g')
axis.set_xlabel(u"分数")
axis.set_ylabel('占比')
plt.title("分数分布")
plt.show()

普通的LR 模型无法归一化，同样的步骤 但是权值和预测结果都很大

In [ ]:
# linear_model = LinearRegression(normalize=True)
# LR_model = linear_model.fit(X_train, y)
# coef = pd.Series(LR_model.coef_, index = X_train.columns)             # .coef_ 可以返回经过学习后的所有 feature 的参数。
# print("LR picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
# for c in coef.index:
#     print(c + " coef :" + str(coef[c]))
# imp_coef = pd.concat([coef.sort_values().head(12), 
#                      coef.sort_values().tail(12)])                       #选头尾各10条，.sort_values() 可以将某一列的值进行排序。
# # matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
# plt.figure(figsize=(20,15))
# imp_coef.plot(kind = "barh")
# plt.title("Coefficients in the LR Model")   
# predict_res = LR_model.predict(X_train)
# score_list = list(map(calculate_score, predict_res))

# plt.figure(figsize=(20, 10))    # 指定画布尺寸
# axis = sns.distplot(a=score_list, color='b')
# axis.set_xlabel(u"分数")
# axis.set_ylabel('占比')
# plt.title("分数分布")
# plt.show()

xgbdt的模型比线性回归表现好一点，前者分数0.182 后者0.192

In [ ]:
linear_model = LinearRegression(normalize=True)
ridge_model = Ridge()
gbdt_model = GradientBoostingRegressor()
rf_model = RandomForestRegressor()
svr_model = SVR(gamma=0.01)
xgb_model = XGBRegressor()

# models = [linear_model,ridge_model,gbdt_model,rf_model,svr_model,xgb_model]
# models_names = ['linear_model','ridge_model','gbdt_model','rf_model','svr_model','xgb_model']
# models_scores_mean = []
# st_kflod = StratifiedKFold(n_splits=10,shuffle=False,random_state=0)
# for index,model in enumerate(models):
#     score = cross_val_score(estimator=model,X=X_train,y=y,scoring='neg_mean_squared_error',cv=10)
#     models_scores_mean.append(-score.mean())
#     print(f'{models_names[index]}'+'分数:' + f'{score}'+'平均分:'+f'{-score.mean()}')
#     print('-'*50)

In [ ]:
data_ino.项目名称.value_counts()

In [ ]:
age = data_ino.groupby("病人ID")["年龄"].min()
# age = age.value_counts()
# age = age.unstack()
# type(age)

# data.top(5)

In [ ]:
# DataFrame(data,index=[0])

# data["体检总结"]

# data.to_pickle('diag_all.pkl')
check = data.groupby("病人ID")['诊断类型'].value_counts()
# check.fillna(0)
check_res = check.unstack().fillna(0) #.describe()
# check_res.apply(lambda x: x['label'] = 0 if x['疾病诊断'] > 0 else 1)
check_res['label'] = np.where(check_res['阳性体征']==0, 0, 1)

In [ ]:
check_res

随机病人ID，检查data_ino和data，分别是检查结果和诊断结果

In [ ]:
ID = tuple(data_ino.病人ID)
sample_ID = random.sample(ID, 1)
# sample = random.sample(data, 5)
sample_ID

# sample_ID
# sample_ID = list(sample_ID)
sample_diag = data[data.病人ID.isin(sample_ID)]
sample_diag
sample_ino = data_ino[data_ino.病人ID.isin(sample_ID)]

sample_ino.异常提示.value_counts()
sample_ino
sample_ino[-((sample_ino.异常提示.isnull()) & (sample_ino.阳性标志.isnull()) & (sample_ino.诊断标志.isnull()))]